In [11]:
import torch
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from transformers import AutoTokenizer, PreTrainedTokenizerFast
from dictionary_learning import AutoEncoder
from nnsight import LanguageModel
from dictionary_learning.utils import read_csv
from IPython.display import display, HTML

In [36]:
import scipy.sparse
from dictionary_learning.sparse_feature_writer import SparseFeatureWriter
from scipy.sparse import csr_matrix
import gc
import h5py

In [13]:
# load autoencoder
ae = AutoEncoder.from_pretrained("/gpfs/helios/home/jpauklin/dictionary_learning/saes/trainer_0/ae.pt").to("cuda") # to is rquired to load to GPU

device = "cuda:0" #GPU
# Load Model
model_name = "/gpfs/space/projects/stacc_health/gpt2_model/estMed-gpt2_fine_tuned4/estMed-gpt2_fine_tuned4"
model = LanguageModel(
    model_name,
    device_map=device,
)

activation_dim = model.transformer.h[0].ln_1.normalized_shape[0] # output dimension of the MLP = 768

/gpfs/helios/home/jpauklin/dictionary_learning/dictionary_learning/dictionary.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = t.load(path)


In [14]:
data = read_csv("/gpfs/space/projects/stacc_health/data-synthetic/100k_synthetic_texts.csv", 16) # (csv_path, nr_of_text_batches_to_read)
text = list(data) # list from iterator

# TODO: Sparse matrix 

# TODO: praegu väheste andmetega korraga. 64 batchi korral OutOfMemory

In [15]:
#text.append("See on test lause, mis on käsitsi lisatud.")

In [16]:
tokenizer_filepath = "/gpfs/space/projects/stacc_health/gpt2_model/estMed-gpt2_fine_tuned4/estMed-gpt2_fine_tuned4/tokenizer.json"
tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_filepath)
tokenizer.pad_token = "<pad>"

# Padding to have batched tensors with the same length.    
tokens = tokenizer(text, return_tensors="pt", padding=True)   # , truncation=True


In [17]:
"""
Idee
Ükshaaval jälgida aktivatsioone. anda sisend üks batch haaval ette. Salvestada features (1 batchi kohta korraga) faili. 

Siht. Meil on kogu andmete kohta featurid. Saame otsida tugevaid aktivatsioone üle kõigi tokenite. 
Saame kindla featuri kõik tugevaimad aktivatsioonid. 

Mis faili formaat? txt või csv 
Failis on batchid eraldatud üksteisest ridade kaupa.

Näiteks leitakse tugev aktivatsioon ühes batchis. Lihtsaim viis teha, et terve batch loetakse ja ss kui on akt 
salvestatakse batch tulemusse. 
"""

'\nIdee\nÜkshaaval jälgida aktivatsioone. anda sisend üks batch haaval ette. Salvestada features (1 batchi kohta korraga) faili. \n\nSiht. Meil on kogu andmete kohta featurid. Saame otsida tugevaid aktivatsioone üle kõigi tokenite. \nSaame kindla featuri kõik tugevaimad aktivatsioonid. \n\nMis faili formaat? txt või csv \nFailis on batchid eraldatud üksteisest ridade kaupa.\n\nNäiteks leitakse tugev aktivatsioon ühes batchis. Lihtsaim viis teha, et terve batch loetakse ja ss kui on akt \nsalvestatakse batch tulemusse. \n'

In [18]:
writer = SparseFeatureWriter("features/features.h5")

In [34]:
for text_batch in text:
    tokens = tokenizer(text_batch, return_tensors="pt", padding=True)
    with model.trace(tokens['input_ids'], tokens.get('attention_mask')) as tracer:
    
        # Selecting a specific layer to capture
        mlp_output = model.transformer.h[11].mlp.c_proj.output.save()
    
    # The model is actually run upon exiting the tracing context. (https://nnsight.net/notebooks/tutorials/walkthrough/)
    activations = mlp_output.value
    features = ae.encode(activations)[0]
    csr_batch = csr_matrix(features.detach().cpu().numpy())
    writer.append(csr_batch)

    torch.cuda.empty_cache() # Free up space
    gc.collect()

In [38]:
with h5py.File("features/features.h5", 'r') as f:
    shape = f.attrs['shape']
    full = csr_matrix((f['data'][:], f['indices'][:], f['indptr'][:]), shape=shape) # Create sparse row matrix from file

In [46]:
with h5py.File("features/features.h5", 'r') as f:
    print(list(f.keys()))

['data', 'indices', 'indptr']


In [9]:
# Using nnsight, we hook into one of the MLP's layer and inspect them during a forward pass.
# This gets us the layer's activations (output of c_proj)

# The "with" keyword starts a context-like object. It defines that we will run the model with tracing. 
with model.trace(tokens['input_ids'], tokens.get('attention_mask')) as tracer:
    
    # Selecting a specific layer to capture
    mlp_output = model.transformer.h[11].mlp.c_proj.output.save()
    
# The model is actually run upon exiting the tracing context. (https://nnsight.net/notebooks/tutorials/walkthrough/)
activations = mlp_output.value  # shape: (batch_size aka nr_of_inputs, seq_len , hidden_dim_activation_value)

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


In [10]:
features = ae.encode(activations) # get features from activations
reconstructed_activations = ae.decode(features)

In [ ]:
activations

In [ ]:
reconstructed_activations

In [ ]:
features.size()

In [ ]:
type(features[0][0][0])

In [ ]:
def token_top_features(tokens, features, batch_idx):
    # features shape: (batch_size, seq_len, num_features)
    feature_activations_for_seq = features[batch_idx]

    # Get corresponding tokens
    input_tokens = tokens['input_ids'][batch_idx]

    # For each token
    for token_idx, token in enumerate(input_tokens):
        decoded_token = tokenizer.decode(token)
        print(f"Token: {decoded_token}")
        feature_acts = feature_activations_for_seq[token_idx]

        # Show top k most active features for token
        topk = torch.topk(feature_acts, k=5)
        print(f"  Top features: {topk.indices.tolist()} with activations {topk.values.tolist()}")

In [ ]:
token_top_features(tokens, features, 0)

In [ ]:
# TODO: Funktsioon, mis leiab harva (kuid tugevasti) käivituvad feature'id.

In [ ]:
# given a specific feature and a batch of input text, outputs the feature's activation value for each token.

def feature_activations(tokens, features, batch_idx, feature_idx):
    # features shape: (batch_size, seq_len, num_features)
    feature_activations_for_seq = features[batch_idx]  # shape: (seq_len, num_features)

    # Get corresponding tokens
    input_tokens = tokens['input_ids'][batch_idx]

    # Print given feature's activation value for each token
    print(f"Feature: {feature_idx}")
    for token_idx, token in enumerate(input_tokens):
        decoded_token = tokenizer.decode(token)
        #token_clean = decoded_token.replace("Ġ", "")
        feature_acts = features[batch_idx][token_idx]
        print(f"{decoded_token: >20} = {feature_acts[feature_idx]}")

In [ ]:
feature_activations(tokens, features, 0, 2492)

In [ ]:
def visualize_feature_activations(tokens, features, batch_idx, feature_idx):
    # detach from computation graph. Move to CPU memory, since numpy only works on CPU tensors.
    feature_activations = features[batch_idx, : , feature_idx].detach().cpu().numpy()

    # Get corresponding tokens
    input_tokens = tokens['input_ids'][batch_idx]
    
    # Color mapping
    max_val = 30 
    min_val = 0
    forward = lambda x: x** (1/3) # function. Transforms the data values before applying the colormap.
    inverse = lambda x: x**3 # function. Transforms from the normalized scale [0, 1] back to the original data space.
    norm = matplotlib.colors.FuncNorm((forward, inverse), vmin=min_val, vmax=max_val) #(https://matplotlib.org/stable/users/explain/colors/colormapnorms.html)
    #norm = matplotlib.colors.Normalize(vmin=min_val, vmax=max_val)
    cmap = matplotlib.colormaps.get_cmap('Oranges')

    # Build HTML with color highlighting
    html = ""
    for token, activation in zip(input_tokens, feature_activations): 
        decoded_token = tokenizer.decode(token)
     
        # TODO: Ei lisaks spani padding tokenite korral
    
        color = matplotlib.colors.rgb2hex(cmap(norm(activation)))
        html += f'<span style="background-color:{color}; padding: 2px; margin: 2px; border-radius: 3px;">{decoded_token}</span> '

    display(HTML(html))

In [ ]:
visualize_feature_activations(tokens, features, 0, 10684)

In [ ]:
# Note: Teha suur tulemustabel/fail, kus saab otsida, feature'eid, mis harva aktiveeruvad või tokenite järgi (terviseterminid)